In [ ]:

from datetime import datetime


# from preauth_provider_side_crew.crew import PreauthProviderSideCrew

# warnings.filterwarnings("ignore", category=SyntaxWarning, module="pysbd")

In [ ]:
inputs = {
        "patient_info": {
            "first_name": "Rajesh",
            "last_name": "Kumar",
            "dob": "1980-05-12",
            "gender": "Male",
            "contact": {
                "phone": "+91-9876543210",
                "email": "rajesh.kumar@example.com"
            },
            "address": "Hyderabad, Telangana, India",
            "ehr_id": "EHR-998877",
            "physician": {
                "name": "Dr. Anjali Mehta",
                "department": "Orthopedics",
                "npi": "1234567890"
            }
        },
        "insurance_card": {
            "insurer_name": "Star Health Insurance",
            "policy_number": "STAR-HI-20394857",
            "group_number": "GRP-112233",
            "valid_from": "2024-01-01",
            "valid_to": "2025-12-31",
            "network_affiliation": "In-Network"
        },
        "clinical_notes": """
                The patient presents with chronic left knee pain for over 6 months, worsening with movement. 
                X-ray confirms osteoarthritis. Recommended Arthroscopic Knee Surgery intervention with pre/post rehab. 
                No known drug allergies. Pre-op evaluation needed.
            """
        ,
        "pricing_data": {
            "procedure_name": "Arthroscopic Knee Surgery",
            "procedure_code": "29881",
            "procedure_cost": 48000,
            "estimated_length_of_stay": "2 days",
            "pre_post_care_cost": 7500,
            "consumables": 3200,
            "total_estimate": 58700
        },
        "coding_reference": {
            "icd_codes": {
                "M17.12":"Unilateral primary osteoarthritis, left knee",
                "M17.11":"Unilateral primary osteoarthritis, right knee",
                "M17.10":"Unilateral primary osteoarthritis, unspecified knee"
                
            },
            "cpt_codes": {
                "29881": "Arthroscopy, knee, surgical; with meniscectomy (medial OR lateral including any meniscal shaving)",
                "29880": "Arthroscopy, knee, surgical; with removal of loose body or foreign body",
                "29882": "Arthroscopy, knee, surgical; with repair of meniscus (medial OR lateral)"
            }
        }
    }
    


In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task, output_pydantic
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from preauth_provider_side_crew.output_models.pydantic_output_models import (
    PatientInfoModel, ClinicalNotesModel, CodingModel,
    CostEstimateModel, JustificationModel, PreAuthFormModel
)

@CrewBase
class PreauthProviderSideCrew():
    """Crew for handling medical preauthorization workflow."""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def patient_intake_agent(self) -> Agent:
        """Creates the patient intake agent for registration."""
        return Agent(
            config=self.agents_config['patient_intake_agent'], # type: ignore[index]
            verbose=True
        )

    @agent
    def gather_clinical_notes_agent(self) -> Agent:
        """Creates the clinical documentation specialist agent."""
        return Agent(
            config=self.agents_config['gather_clinical_notes_agent'], # type: ignore[index]
            verbose=True
        )

    @agent
    def generate_standard_codes_agent(self) -> Agent:
        """Creates the medical coding specialist agent."""
        return Agent(
            config=self.agents_config['generate_standard_codes_agent'], # type: ignore[index]
            verbose=True
        )

    @agent
    def estimate_treatment_costs_agent(self) -> Agent:
        """Creates the cost estimation analyst agent."""
        return Agent(
            config=self.agents_config['estimate_treatment_costs_agent'], # type: ignore[index]
            verbose=True
        )       
        

    @agent
    def summarize_clinical_justification_agent(self) -> Agent:
        """Creates the clinical justification writer agent."""
        return Agent(
            config=self.agents_config['summarize_clinical_justification_agent'], # type: ignore[index]
            verbose=True
        )

    @agent
    def format_preauth_form_agent(self) -> Agent:
        """Creates the preauthorization document specialist agent."""
        return Agent(
            config=self.agents_config['format_preauth_form_agent'], # type: ignore[index]
            verbose=True
        )
        
    @task
    def process_patient_registration_task(self) -> Task:
        """Task for processing patient registration."""
        return Task(
            config=self.tasks_config['process_patient_registration_task'], # type: ignore[index]
            output_pydantic=PatientInfoModel
        )

    @task
    def extract_clinical_terms_task(self) -> Task:
        """Task for processing clinical notes and documentation."""
        return Task(
            config=self.tasks_config['extract_clinical_terms_task'], # type: ignore[index]
            output_pydantic=ClinicalNotesModel
        )

    @task
    def medical_coding_task(self) -> Task:
        """Task for generating standardized medical codes."""
        return Task(
            config=self.tasks_config['medical_coding_task'], # type: ignore[index]
            output_pydantic=CodingModel
        )

    @task
    def estimate_treatment_costs_task(self) -> Task:
        """Task for estimating treatment costs."""
        return Task(
            config=self.tasks_config['estimate_treatment_costs_task'], # type: ignore[index]
            output_pydantic=CostEstimateModel
        )

    @task
    def summarize_clinical_justification_task(self) -> Task:
        """Task for creating clinical justification summary."""
        return Task(
            config=self.tasks_config['summarize_clinical_justification_task'], # type: ignore[index]
            output_pydantic=JustificationModel
        )

    @task
    def format_preauth_form_task(self) -> Task:
        """Task for formatting preauthorization submission."""
        return Task(
            config=self.tasks_config['format_preauth_form_task'], # type: ignore[index]
            output_pydantic=PreAuthFormModel
        )

    @crew
    def crew(self) -> Crew:
        """Creates the PreauthProviderSideCrew crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
            name="Hospital PreAuthorization Provider-Side Automation"
        )


In [ ]:
try:
    PreauthProviderSideCrew().crew().kickoff(inputs=inputs)
except Exception as e:
    raise Exception(f"An error occurred while running the crew: {e}")